# QCQuant Fitting

We're following Narla *et al.* (PNAS 2021)to come to analytical approximations to the solution of the growth-expansion model of chemotaxis created by Cremer *et al.* (Nature 2020(?)). The Narla work is under very specific conditions that don't always apply. Specifically, they are that 

$$ r \ll \lambda c $$ (1)

where $r$ is the growth rate (per hour), $\lambda$ is the attractant gradient the bacteria follow (*i.e.*, $a(x|t)\propto \exp (\lambda x)$), and $c$ is the speed of the taveling density bulge. This assumption is that growth is much slower than chemotactic drift, and ends up being 

$$ \frac{\lambda}{\lambda_g} \gg 1 $$, (2)

which is to say the attractant slope is much steeper than the growth slope (*n.b.*, I'm not entirely sure this isn't already assuming Eqn. 1. I think it does).

## Useful Equations

Eqn. 12: $\lambda = \frac{c}{\chi_0-D_\rho}$

$c$ is the speed of the chemotaxis wave, $\chi_0$ is the Chemotactic Motility Parameter ($\mu m^2/s$), and $D_\rho$ is the Bacterial Motility Parameter ($\mu m^2/s$).


Eqn. 28: $\lambda_d \approx \frac{c}{D_\rho}$


Eqn. 36: $\lambda_g \approx \frac{r}{c}$

Definition of $z$:
$z \equiv x - ct$

Minimum of Eqn. 38:
$z_{min} = z_m + \frac{1}{\lambda} ln \frac{\lambda_g}{\lambda-\lambda_g}$
I've been defining $\lambda_c \equiv \lambda -\lambda_g$ 

Note that if you're not in the $\lambda \gg \lambda _g $ parameter regime, then $z_m$ can be either to the right or the left of $z_{min}$, so the diagram in Fig. 6a of Narla et al is a little misleading (as is Eqn. 39). Equation 39 comes about because when $\lambda \gg \lambda _g $, $z_{min}\approx z_m + \frac{1}{\lambda} ln \frac{\lambda_g}{\lambda} = z_m - \frac{1}{\lambda} ln \frac{\lambda}{\lambda_g} = z_m - \frac{1}{\lambda} ln \frac{\lambda}{1}\frac{c}{r}$, where the last step uses Eqn. 36.

# My model
Narla et al is difficult to apply and really only has piece-wise descriptions of certain regimes. Notably, it does not have a description of the density bulge. My model pulls together a few of the Narla descriptions and staples them together with a fake peak to cover the full radial density with just a few parameters.

There are four regions: growth, chemotaxis, "peak", and diffusion. Narla et al provide a description of the turnover between growth and chemotaxis, which is just a mixture of two exponentials facing different directions. Therefore, is actually quite a bit past the the turnover region. 

My regions are: growth+chemoaxis, "peak", and diffusion. Also, I place left and right cutoffs on the data to avoid any agar stab wounds and the petri dish edge. Also, all of this takes place on top of a floor (i.e., background, b). 

## growth+chemotaxis
Following Eqn. 38 and expanding for a set time (and also redefining variables), you get

$$ \rho_G(x) = \rho_g \exp \left( -k_g (x-x_g) \right) + \rho_g \exp \left( +k_c (x-x_g) \right) + b,$$

where all $k_i$ are defined as positive (direction defined in equations), and here $k_c\equiv k-k_g$. Note that you should compare this to Eqn. "37.5" which says that in the chemotaxis regime ($a(z) > a_m$), then $\rho(z)=\beta a_m e^{(\lambda -\lambda_g)(z-z_m)}$, i.e., this is where we get $\lambda_c$.

## peak
To cover the peak region, we use a gaussian of the form

$$ \rho(x) = \rho_{peak} e^{-\frac{1}{2\sigma^2_{peak}}(x-x_{d})^2} +b,$$

which is centered at the point $x_d$. Notably, we don't treat $\rho_{peak}$ as an indepdendent variable, but instead calculate it value by matching this equation to the value of $\rho_G(x)$ at $x=x_{switch,l}\equiv x_d-\sigma_{peak}/2$. There exists a similar $x_{switch,r}$ on the other side.

## diffusion
In the diffusion region, we return to the Narla, and use Eqn. 27 which is the solution to the F-KPP equation. Thus, after the cutoff point at $x_{switch,r}$, the density becomes 

$$\rho_D(x) = \rho_d e^{\lambda_d (x-x-d)}+b,$$
where this decays starts at the density peak. Practically, we turn this decay on after the right switch point by matching $\rho_d$ to the value of the peak density at the point. (n.b., maybe a +1 indexing error in my code here...).

# Diffusion only model
we modify this model for finite capacity diffusion by having a cosntant value $\rho_d$ be the prefactor to a diffusion like decay. Thus, this model looks like a reversed Heaviside step function, but instead of a sharp drop, it falls off with half of the gaussian peak, and then turns into the exponential diffusion decay.

# Code
This is all worked out in qcquant_fitting.py.
For an example use, see `examples/goodpeaks/test.ipynb`